In [1]:
#from google.colab import files

#ploaded = files.upload()

In [2]:
# !pip install fuzzywuzzy
# !pip install python-Levenshtein

In [3]:
import pandas as pd
import re

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [4]:
data = pd.read_excel('/content/Книга1.xlsx')

In [5]:
data.rename(columns={list(data)[0]:'card_num', list(data)[1]:'barcode', list(data)[2]:'description'}, inplace=True)

In [6]:
data['new_description'] = data['description'].copy()

In [7]:
data

,card_num,barcode,description,new_description
0,1,123,Кофе Якобс/ 100гр,Кофе Якобс/ 100гр
1,1,234,Чай Липтон 100гр,Чай Липтон 100гр
2,2,345,Чай Гринфилд 200гр,Чай Гринфилд 200гр
3,2,942,Чай Гринфилд 200гр,Чай Гринфилд 200гр
4,2,456,Чай 500гр,Чай 500гр
5,3,124,Печеньки Юбилейные 500гр,Печеньки Юбилейные 500гр
6,3,125,Печеньки 500гр,Печеньки 500гр
7,3,126,Макароны,Макароны
8,4,645,Чипсы принглс 200гр,Чипсы принглс 200гр
9,4,354,Чипсы принглс 200гр,Чипсы принглс 200гр


In [8]:
# пишем функцию которая преобразовывет description в новый
# description, приводим все названия к нижнему регистру и
# избавляемся от лишних символов и пробелов

def process_description(description):
    cleaned_description = re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', description)
    lowercase_description = cleaned_description.lower()
    trimmed_description = lowercase_description.strip()
    return trimmed_description

# Создаем столбец `new_description` с обработанными данными
data['new_description'] = data['description'].apply(process_description)

In [9]:
# задаем четкие правила, что правильная карточка должна оканчиваться
# на "гр" в противном случае, будет выведен False у поля, где данное
# правило не соблюдено.

def check_string_format(string):
    pattern = r'^\w+(?:\s+\w+)+\s+\d+гр$'
    match = re.match(pattern, string)
    return bool(match)

data['description_check'] = data['new_description'].apply(check_string_format)

In [10]:
# здесь видно, что в столбце description_check стоят значения True или False
# если стоит значение True значит карточка с орфографической точки заполнена правильно
# если False, то либо нет названия бренда, либо не указаны граммовки и тд

data

,card_num,barcode,description,new_description,description_check
0,1,123,Кофе Якобс/ 100гр,кофе якобс 100гр,True
1,1,234,Чай Липтон 100гр,чай липтон 100гр,True
2,2,345,Чай Гринфилд 200гр,чай гринфилд 200гр,True
3,2,942,Чай Гринфилд 200гр,чай гринфилд 200гр,True
4,2,456,Чай 500гр,чай 500гр,False
5,3,124,Печеньки Юбилейные 500гр,печеньки юбилейные 500гр,True
6,3,125,Печеньки 500гр,печеньки 500гр,False
7,3,126,Макароны,макароны,False
8,4,645,Чипсы принглс 200гр,чипсы принглс 200гр,True
9,4,354,Чипсы принглс 200гр,чипсы принглс 200гр,True


In [11]:
# здесь выводит строки, где сотрудник заполнил карточку неправильно

data[data['description_check'] == False]

,card_num,barcode,description,new_description,description_check
4,2,456,Чай 500гр,чай 500гр,False
6,3,125,Печеньки 500гр,печеньки 500гр,False
7,3,126,Макароны,макароны,False


In [12]:
# пишем функцию, которая будет группировать карточки по их номеру,
# а затем сравнивать между собой, построчно внутри группировки
# если совпадение менее 80%, то будет выведенно текстовое предупреждение
# 'Нет ничего общего', если более 80%, то выведен будет процент совпадения
# у обоих значений

def compare_within_card(card_data):

    # Создаем столбик для хранения результатов сравнения
    card_data['similarity_ratio'] = pd.np.nan
    grouped = card_data.groupby('card_num')

    for _, group in grouped:
        for i in range(len(group)):
            for j in range(i + 1, len(group)):
                description1 = group.iloc[i]['new_description']
                description2 = group.iloc[j]['new_description']

                # Вычисляем процент совпадения описаний с помощью fuzzywuzzy
                similarity_ratio = fuzz.ratio(description1, description2)

                if similarity_ratio < 80:
                  card_data.at[group.iloc[i].name, 'similarity_ratio'] = 'Нет ничего общего'
                  card_data.at[group.iloc[j].name, 'similarity_ratio'] = 'Нет ничего общего'
                else:
                  card_data.at[group.iloc[i].name, 'similarity_ratio'] = similarity_ratio
                  card_data.at[group.iloc[j].name, 'similarity_ratio'] = similarity_ratio

    return card_data

In [13]:
data = compare_within_card(data)
display(data)

<ipython-input-12-7b0c226470d6>:10: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  card_data['similarity_ratio'] = pd.np.nan


,card_num,barcode,description,new_description,description_check,similarity_ratio
0,1,123,Кофе Якобс/ 100гр,кофе якобс 100гр,True,Нет ничего общего
1,1,234,Чай Липтон 100гр,чай липтон 100гр,True,Нет ничего общего
2,2,345,Чай Гринфилд 200гр,чай гринфилд 200гр,True,Нет ничего общего
3,2,942,Чай Гринфилд 200гр,чай гринфилд 200гр,True,Нет ничего общего
4,2,456,Чай 500гр,чай 500гр,False,Нет ничего общего
5,3,124,Печеньки Юбилейные 500гр,печеньки юбилейные 500гр,True,Нет ничего общего
6,3,125,Печеньки 500гр,печеньки 500гр,False,Нет ничего общего
7,3,126,Макароны,макароны,False,Нет ничего общего
8,4,645,Чипсы принглс 200гр,чипсы принглс 200гр,True,100
9,4,354,Чипсы принглс 200гр,чипсы принглс 200гр,True,100
